##### Set up packages and directories

In [1]:
full_run = True

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
import math
from tqdm.auto import tqdm
from Comparative_Analysis import Sequence_Analysis_Routines as sar
from Comparative_Analysis import Utilities as util
from Comparative_Analysis import Alignment as align
import random
import copy
from joblib import Parallel, delayed
import os
import shutil
import subprocess
from Bio.Align.Applications import MuscleCommandline
muscle_exe = 'C:/Users/nicho/Muscle/muscle3.8.31_i86win32.exe'

In [3]:
project_dir = 'F:/Project_Data/Self_Blast_Project'
literature_datasets_dir = 'F:/Datasets/Data_From_Publications'
output_dir = project_dir + '/Output'
refseq_dir = project_dir + '/Datasets'
num_cores = 8
core_numbers = list(range(1, num_cores+1))

In [4]:
reference_species = 'GCF_000195955.2'
all_species = util.list_dirs(refseq_dir)

In [5]:
rbh = pd.read_csv('F:/Project_Data/Intergenic_Region_Comparative_Analysis/reciprocal_best_hits.csv')
rbh_dict = {}
for i, r in rbh.iterrows():
    rbh_dict[r['target_ref']] = r['query_ref']

In [6]:
annotated_regions_dict = {}
for species in tqdm(all_species):
    for record in SeqIO.parse(refseq_dir + '/'+species+'/genomic.gbff', "genbank"):
        annotated_regions = []
        intergenic_regions = []
        accession_ver = record.annotations['accessions'][0] + '.' + str(record.annotations['sequence_version'])
        for feature in record.features:
            a = feature.qualifiers
            if feature.type not in ['source','gene'] and (int(feature.location.start) < int(feature.location.end)) and (int(feature.location.end) - int(feature.location.start)) < 1000000:
                if not(a.get("product") == None):
                       product = a.get("product")[0]
                else:
                       product = feature.type
                if not(a.get("locus_tag")==None):
                    locus_tag = accession_ver + '@' + a.get("locus_tag")[0]
                    if locus_tag in rbh_dict:
                        ortholog_locus_tag = rbh_dict[locus_tag]
                    else:
                        ortholog_locus_tag = ''
                else:
                    locus_tag = feature.type
                    ortholog_locus_tag = ''
                annotated_regions.append((locus_tag, ortholog_locus_tag, product, feature.type, int(feature.location.start), int(feature.location.end)))
        annotated_regions.sort(key = lambda x: x[4])
        prev_locus = ''
        prev_ortholog_locus = ''
        prev_product = ''
        max_stop = 0
        for n, (locus, ortholog_locus, product, feature_type, start, stop) in enumerate(annotated_regions):
            if start > max_stop:
                intergenic_regions.append([prev_locus+':'+locus, prev_ortholog_locus + ':' + ortholog_locus, prev_product + ':' + product, 'Inter-feature',max_stop, start])
            if stop > max_stop:
                prev_locus = locus
                prev_ortholog_locus = ortholog_locus
                prev_product = product
            max_stop = max(max_stop, stop)    
        for x in intergenic_regions:
            annotated_regions.append(x)
        annotated_regions.sort(key = lambda x : x[4])
        annotated_regions_dict[accession_ver] = annotated_regions

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
if 1== 0:
    with open(project_dir + '/' + 'annotated_regions_dict.pkl', 'wb') as f:
        pickle.dump(annotated_regions_dict, f)
else:
    with open(project_dir + '/' + 'annotated_regions_dict.pkl', 'rb') as f:
        annotated_regions_dict = pickle.load(f)    

In [22]:
full_sequence_dir_dict = {}
species_dir_dict = {}
for spec in tqdm(all_species):
    for record in SeqIO.parse(refseq_dir + '/'+spec+'/genomic.gbff', "genbank"):
        accession_ver = record.annotations['accessions'][0] + '.' + str(record.annotations['sequence_version'])
        full_sequence = str(record.seq)
        species_dir_dict[accession_ver] = spec
        full_sequence_dir_dict[accession_ver.replace('_','')] = full_sequence

  0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
min_nts = 30

In [23]:
def produce_full_sequence_and_intergenic_region_fasta(refseq_dir, species, search_against_multiple_species = False, multiple_species = []):
    loci = []
    record = next(SeqIO.parse(refseq_dir + '/'+species+'/genomic.gbff', "genbank"))
    accession_ver = record.annotations['accessions'][0] + '.' + str(record.annotations['sequence_version'])     #  for this version just use first record in annotation
    full_sequence = str(record.seq)
    for feature in record.features:
            a = feature.qualifiers
            if feature.type not in ['source','gene'] and (int(feature.location.start) < int(feature.location.end)) and (int(feature.location.end) - int(feature.location.start)) < 1000000:
                loci.append((feature.type, int(feature.location.start), int(feature.location.end)))

    loci.sort(key = lambda x: x[1])
    std_annotation_intergenic_regions = []
    max_stop = 0
    for (feature_type, start, stop) in loci:
        if start > max_stop + min_nts:
            std_annotation_intergenic_regions.append([accession_ver.replace('_','') + '_'+ str(max_stop)+'_'+str(start), full_sequence[max_stop: start]])
        max_stop = max(max_stop, stop)
    util.produce_fasta_file(std_annotation_intergenic_regions, project_dir + '/'+ species+'_intergenic_regions.faa')
    
    #  Produce fasta file will full sequences for all species for blasting against
    if search_against_multiple_species == True:
        search_species = multiple_species
    else:
        search_species = [species]
    temp = []
    for spec in search_species:
        record = next(SeqIO.parse(refseq_dir + '/'+spec+'/genomic.gbff', "genbank"))
        accession_ver = record.annotations['accessions'][0] + '.' + str(record.annotations['sequence_version'])
        full_sequence = str(record.seq)
        temp.append([accession_ver, full_sequence])
    util.produce_fasta_file(temp, project_dir + '/full_'+species+'_sequence.fasta')



In [19]:
def make_blast_sequence_db(species):
    w_d = os.getcwd()
    os.chdir("F:/")
    subprocess.run('cd '+ project_dir + ' &  makeblastdb -in ' + project_dir + '/full_'+species+'_sequence.fasta' +' -dbtype nucl -out full_'+species+'_sequence_nt', shell=True, capture_output = True)
    os.chdir(w_d)
    if not(os.path.exists('F:/Datasets/BLAST/Self_BLAST/' + species)):
        os.makedirs('F:/Datasets/BLAST/Self_BLAST/' + species)
    files = util.list_files(project_dir)
    for file in files:
        if species in file:
            shutil.move(project_dir+'/'+file, 'F:/Datasets/BLAST/Self_BLAST/' + species +'/' + file)  

In [32]:
def run_self_blast(refseq_dir, species):
    w_d = os.getcwd()
    os.chdir("F:/")
    subprocess.run('cd f:\\Datasets\\BLAST\\Self_BLAST\\' + species + ' & blastn -query ' + species + '_intergenic_regions.faa -db full_'+species+'_sequence_nt -out hits.csv -evalue 10 -outfmt "10 qaccver saccver qlen slen pident length mismatch gapopen qstart qend sstart send evalue bitscore" -num_threads 16', shell=True, capture_output = True)
  
    os.chdir(w_d)
    blast_results = pd.read_csv('F:/Datasets/BLAST/Self_BLAST/' + species + '/hits.csv', header = None)
    blast_results.columns = ['query_ref', 'target_ref', 'query_length', 'subject_length', 'percent_identical_matches','alignment_length', 'number_mismatches', 'number_of_gap_openings', 
                             'query_start_alignment', 'query_end_alignment', 'target_start_alignment', 'target_end_alignment', 'e_value', 'bit_score']
    
    blast_results['hit_count'] = blast_results.groupby('query_ref')['target_ref'].transform('count')
    blast_results = blast_results[blast_results['hit_count'] > 1]
    
    repeat_regions = []
    blast_results['annot_features']=''
    for i, r in blast_results.iterrows():
        start1 = min(r['target_start_alignment'],r['target_end_alignment'])
        end1 = max(r['target_start_alignment'],r['target_end_alignment'])
        feature_matches = []
        for (locus, ortholog_locus, product, feature, start, stop) in annotated_regions_dict[r['target_ref']]:
            if start< end1 and stop > start1:
                overlap = (min(end1, stop) - max(start1, start))/ (end1-start1)
                #Don't want to output ortholog info as it clutters!
                #feature_matches.append([locus, ortholog_locus, product, feature, overlap])
                feature_matches.append([locus, product, feature, overlap])
                if ('repeat' in feature) or ('mobile' in feature):
                    repeat_regions.append(r['query_ref'])
        blast_results.at[i,'annot_features'] = feature_matches
    
    
    # remove annotated repeat regions
    repeat_regions = list(set(repeat_regions))
    blast_results = blast_results.query("not(query_ref.isin(@repeat_regions))")
    
    blast_results['ref_string'] = ''
    blast_results['ref_start'] = 0
    blast_results['ref_stop'] = 0
    blast_results['ref_sequence'] = 0
    for i, r in blast_results.iterrows():
        temp = ''
        for l1 in r['annot_features']:
            temp += str(l1[0])
        blast_results.at[i, 'ref_string'] = temp
        blast_results.at[i, 'accession_ver'] = r['query_ref'].split('_')[0]
        blast_results.at[i, 'ref_start'] = int(r['query_ref'].split('_')[1])
        blast_results.at[i, 'ref_stop'] = int(r['query_ref'].split('_')[2])
        
        target_start_alignment = int(r['target_start_alignment'])
        target_end_alignment = int(r['target_end_alignment'])
        if target_start_alignment < target_end_alignment:
            blast_results.at[i, 'ref_sequence'] = full_sequence_dir_dict[r['query_ref'].split('_')[0]][target_start_alignment:target_end_alignment]
        else:
            blast_results.at[i, 'ref_sequence'] = util.reverse_complement(full_sequence_dir_dict[r['query_ref'].split('_')[0]][target_end_alignment:target_start_alignment])
    blast_results = blast_results.loc[blast_results.groupby(['query_ref','ref_string'])['bit_score'].idxmax()]
    blast_results = blast_results.drop_duplicates(['query_ref','ref_string'])
    blast_results['hit_count'] = blast_results.groupby('query_ref')['target_ref'].transform('count')
    blast_results = blast_results[blast_results['hit_count'] > 1]
    blast_results.sort_values(by=['ref_start', 'bit_score'], inplace = True, ascending = [True, False])
    
    blast_results.drop(columns=['accession_ver', 'ref_start', 'ref_stop', 'ref_string'], inplace = True)
    
    #blast_results['tb_count'] = blast_results[blast_results['target_ref'] == 'NC_000962.3'].groupby('query_ref')['target_ref'].transform('count')
    #blast_results['tb_count'] = blast_results.groupby('query_ref')['tb_count'].transform('max')
    #blast_results['bovis_count'] = blast_results[blast_results['target_ref'] == 'LT708304.1'].groupby('query_ref')['target_ref'].transform('count')
    #blast_results['bovis_count'] = blast_results.groupby('query_ref')['bovis_count'].transform('max')
    
    
    
    blast_results.to_excel('F:/Datasets/BLAST/Self_BLAST/' + species + '/processed_hits.xlsx', sheet_name = 'Sheet_1', index = False)

##### Do self hits for each species - request 1

In [33]:
for species in tqdm(all_species):
    produce_full_sequence_and_intergenic_region_fasta(refseq_dir, species, False)
    make_blast_sequence_db(species)
    run_self_blast(refseq_dir, species)

  0%|          | 0/4 [00:00<?, ?it/s]


100%|██████████| 2216/2216 [00:00<00:00, 82659.61it/s]

100%|██████████| 1/1 [00:00<00:00,  8.90it/s]

100%|██████████| 3415/3415 [00:00<00:00, 94976.81it/s]

100%|██████████| 1/1 [00:00<00:00,  8.43it/s]

100%|██████████| 3221/3221 [00:00<00:00, 73769.91it/s]

100%|██████████| 1/1 [00:00<00:00,  8.25it/s]

100%|██████████| 2216/2216 [00:00<00:00, 85301.09it/s]

100%|██████████| 1/1 [00:00<00:00, 12.22it/s]


#####  Do hits to tb and bovis against tb regions - request 2

In [110]:
tb_and_bovis_species = ['GCF_000195955.2','LT708304.1']
tb_species = 'GCF_000195955.2'

produce_full_sequence_and_intergenic_region_fasta(refseq_dir, tb_species, True, tb_and_bovis_species)
make_blast_sequence_db(tb_species)
run_self_blast(refseq_dir, tb_species)

100%|██████████| 2/2 [00:00<00:00, 10.68it/s]


In [112]:
for record in SeqIO.parse(refseq_dir + '/' + reference_species +'/genomic.gbff', "genbank"):
    full_sequence = str(record.seq)

In [113]:
full_sequence[336310:336559]

'CTGACACCTCCCAATACGCATGACCGCTCTGTCATGCCGACCCGGGGAACGTCACCAGCAAAAATCGGCAGTAAGAAGCATCCCATTTCCAGCGACAACACCTGGGGGGTTTTGGTCAAACTCTGGTAAGCGACTTCGTGTACCGGGTGAACCCGGTGTGTCTTGAAGGACAGCCCGCAGGCTGATGCTGGGGGATCTGGGCCGGCCGACCATGGCTGGCCGGCTGTTGGTCTGATGGCCGGTTCGCGG'

In [35]:
annotated_regions_dict = {}
for species in (all_species):
    for record in SeqIO.parse(refseq_dir + '/'+species+'/genomic.gbff', "genbank"):
         organism = record.annotations['organism']
    print(species, organism)

GCF_000195955.2 Mycobacterium tuberculosis H37Rv
GCF_013349145.1 Mycolicibacterium smegmatis
GCF_016745295.1 Mycobacterium marinum
LT708304.1 Mycobacterium tuberculosis variant bovis AF2122/97
